# Two Stage Least Squares Regression
## Addressing Endogeneity 

In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv('../data/2sls.csv')
df.head()

,Education,Libraries_Per_Capita,Experience,Job_Performance
0,7.791772,0.787225,12.941173,103.609460
1,18.419382,1.489825,7.015658,106.494572
2,19.157024,0.928640,19.629696,149.196205
3,17.962375,1.305998,10.749374,115.039354
4,15.783162,0.963582,12.085993,118.516198


In [6]:
y = df['Job_Performance']
X = df[['Education','Experience']]
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
edu_coef_ols = results.params[1]
results.summary()
edu_coef_ols

/var/folders/mq/k4my6f796832wdwl98qv078w0000gn/T/ipykernel_13217/1988907980.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edu_coef_ols = results.params[1]


2.0140939655150487

In [8]:
#Stage 1 
stage_1 = sm.OLS(df['Education'], sm.add_constant(df['Libraries_Per_Capita'])).fit()
df['Education_2'] = stage_1.predict()

#Stage 2
stage_2 = sm.OLS(df['Job_Performance'], sm.add_constant(df[['Education_2', 'Experience']])).fit()
edu_coef_2sls = stage_2.params[1]
stage_2.summary()

/var/folders/mq/k4my6f796832wdwl98qv078w0000gn/T/ipykernel_13217/336898344.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edu_coef_2sls = stage_2.params[1]


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Job_Performance   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     2746.
Date:                Fri, 07 Jun 2024   Prob (F-statistic):               0.00
Time:                        12:08:19   Log-Likelihood:                -3270.8
No. Observations:                1000   AIC:                             6548.
Df Residuals:                     997   BIC:                             6562.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          75.3248     81.160      0.928      0.354     -83.939     234.589
Education_2     0.4023      5.101      0.079      0.937      -9.608      10.412
Experience      3.0143      0.041     74.107      0.000       2.934       3.094
==============================================================================
Omnibus:                        0.449   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.799   Jarque-Bera (JB):                0.330
Skew:                          -0.003   Prob(JB):                        0.848
Kurtosis:                       3.089   Cond. No.                     7.67e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.67e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
overstated_impact = edu_coef_ols - edu_coef_2sls
print(overstated_impact)

1.6117924767220448
